In [1]:
from pprint import pprint
import sys
sys.path.append('../')

from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer

### MaxScoreTokenizer

아래 문장을 단어들로 직접 나눠보세요

    이런문장을직접토크나이징을해볼게요
    
우리는 지금 토크나이징을 이야기 하고 있기 때문에, '토크나이징'이라는 단어가 눈에 잘 들어온다. 그 다음으로는 '문장', '직접', '볼게요' 순으로 단어가 눈에 들어온다 (고 가정하자). 그렇다면 아래 순서대로 단어에 마킹을 할 수 있다. 

    이런문장을직접[토크나이징]을해볼게요
    이런[문장]을직접[토크나이징]을해볼게요
    이런[문장]을[직접][토크나이징]을해볼게요
    이런[문장]을[직접][토크나이징]을해[볼게요]

단어로 인식되지 않은 부분들은 그대로 이어서 하나의 단어로 취급하자. 

    [이런][문장][을][직접][토크나이징][을해][볼게요]

그 결과 [이런, 문장, 을, 직접, 토크나이징, 을해, 볼게요]라는 토큰을 얻게 된다. 아쉬운 점은 [토크나이징, 을, 해, 볼게요]이지만, -을, 해-를 제대로 인식하지 못하는 문제는 있다. 이는 나중에 다른 방식의 토크나이저에서 해결하자. 더 중요한 점은, 단어라고 확신이 드는 부분부터 연속된 글자집합에서 잘라내어도 토크나이징이 된다는 점이다. 


단어 추출 기법을 통하여 subword 들의 단어 가능 점수를 계산할 수 있다. 우리에게 아래의 네 가지 subwords의 점수표와 예문이 있다고 하자. 

    파스타가좋아요
    
    scores = {'파스': 0.3, '파스타': 0.7, '좋아요': 0.2, '좋아':0.5}
    
단어 길이의 범위를 [2, 3]이라고 가정하면 아래와 같은 subword score를 얻을 수 있다. 아래는 (subword, begin, end, score)이다. 

    [('파스', 0, 2, 0.3),
     ('파스타', 0, 3, 0.7),
     ('스타', 1, 3, 0),
     ('스타가', 1, 4, 0),
     ('타가', 2, 4, 0),
     ('타가좋', 2, 5, 0),
     ('가좋', 3, 5, 0),
     ('가좋아', 3, 6, 0),
     ('좋아', 4, 6, 0.5),
     ('좋아요', 4, 7, 0.2),
     ('아요', 5, 7, 0)]
     
이를 점수 순서로 정렬하면 아래와 같다. 

    [('파스타', 0, 3, 0.7),
     ('좋아', 4, 6, 0.5),
     ('파스', 0, 2, 0.3),
     ('좋아요', 4, 7, 0.2),
     ('스타', 1, 3, 0),
     ('스타가', 1, 4, 0),
     ('타가', 2, 4, 0),
     ('타가좋', 2, 5, 0),
     ('가좋', 3, 5, 0),
     ('가좋아', 3, 6, 0),
     ('아요', 5, 7, 0)]

파스타라는 subword의 점수가 가장 높으니, 이를 토큰으로 취급하고, 파스타의 범위인 [0, 3)과 겹치는 다른 subwords을 리스트에서 지워주면 아래와 같은 토큰 후보들이 남는다. 

    파스타가좋아요 > [파스타]가좋아요
    
    [('좋아', 4, 6, 0.5),
     ('좋아요', 4, 7, 0.2),
     ('가좋', 3, 5, 0),
     ('가좋아', 3, 6, 0),
     ('아요', 5, 7, 0)]

다음으로 '좋아'를 단어로 인식하면 남은 토큰 후보가 없기 때문에 아래처럼 토크나이징이 되며, 남는 글자들 역시 토큰으로 취급하면 토크나이징이 끝난다. 

    파스타가좋아요 > [파스타]가[좋아]요 > [파스타, 가, 좋아, 요]
    
단어 점수만을 이용하여도 손쉽게 토크나이징을 할 수 있다. 이 방법의 장점은 각 도메인에 적절한 단어 점수를 손쉽게 변형할 수 있다는 점이다. 도메인에서 반드시 단어로 취급되어야 하는 글자들이 있다면, 그들의 점수를 scores에 최대값으로 입력하면 된다. Score tie-break는 글자가 오버랩이 되어 있다면, 좀 더 긴 글자를 선택하자. 그럼 합성명사 역시 처리하기 쉽다. 

    scores = {'서울': 1.0, '대학교': 1.0, '서울대학교': 1.0} 
    
위처럼 단어 점수가 부여된다면 '서울대학교'를 [서울, 대학교]로 분리하지는 않을 것이다. 대신 '서울'이나 '대학교'가 등장한 다른 어절에서는 이를 단어로 분리한다. 

MaxScoreTokenizer는 이러한 컨셉으로, 단어 점수를 토크나이저에 입력하여 원하는 단어를 잘라낸다. 이는 띄어쓰기가 제대로 이뤄지지 않은 텍스트를 토크나이징하기 위한 방법이며, 단어 점수를 잘 정의하는 것은 단어 추출의 몫이다. MaxTokenizer의 사용법은 아래와 같다. MaxScoreTokenizer를 생성할 때, scores에 {str:float} 형태의 단어 점수 사전을 입력한다.  

In [2]:
scores = {'파스': 0.3, '파스타': 0.7, '좋아요': 0.2, '좋아':0.5}

tokenizer = MaxScoreTokenizer(scores=scores)
tokenizer.tokenize('파스타가좋아요')

['파스타', '가', '좋아', '요']

띄어쓰기가 포함되어 있는 문장은 띄어쓰기를 기준으로 토큰을 나눈 뒤, 그 안에서 토크나이저가 작동한다. flatten을 하지 않으면 list of list 형식으로 토크나이징 결과가 출력된다. list of list에는 (subword, begin, end, score, length)가 저장되어 있다. Default value는 flatten=True이다. 

In [3]:
print('flatten=False')
pprint(tokenizer.tokenize('난파스타가 좋아요', flatten=False))

print('\nflatten=True')
pprint(tokenizer.tokenize('난파스타가 좋아요'))

flatten=False
[[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3), ('가', 4, 5, 0.0, 1)],
 [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]

flatten=True
['난', '파스타', '가', '좋아', '요']


## LTokenizer

띄어쓰기가 잘 되어 있는 한국어 문서의 경우에는 MaxScoreTokenizer를 이용할 필요가 없다. 한국어는 L+[R] 구조이기 때문이다 (단어 추출 튜토리얼 참고). 이 때에는 한 어절의 왼쪽에서부터 글자 점수가 가장 높은 부분을 기준으로 토크나이징을 한다.  

In [4]:
scores = {'데이':0.5, '데이터':0.5, '데이터마이닝':0.5, '공부':0.5, '공부중':0.45}

tokenizer = LTokenizer(scores=scores)

print('\nflatten=True\nsent = 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다'))

print('\nflatten=False\nsent = 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터마이닝을 공부한다', flatten=False))

print('\nflatten=False\nsent = 데이터분석을 위해서 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터분석을 위해서 데이터마이닝을 공부한다', flatten=False))

print('\nflatten=True\nsent = 데이터분석을 위해서 데이터마이닝을 공부한다')
print(tokenizer.tokenize('데이터분석을 위해서 데이터마이닝을 공부한다'))


flatten=True
sent = 데이터마이닝을 공부한다
['데이터마이닝', '을', '공부', '한다']

flatten=False
sent = 데이터마이닝을 공부한다
[('데이터마이닝', '을'), ('공부', '한다')]

flatten=False
sent = 데이터분석을 위해서 데이터마이닝을 공부한다
[('데이터', '분석을'), ('위해서', ''), ('데이터마이닝', '을'), ('공부', '한다')]

flatten=True
sent = 데이터분석을 위해서 데이터마이닝을 공부한다
['데이터', '분석을', '위해서', '데이터마이닝', '을', '공부', '한다']


Tolerance는 한 어절에서 subword 들의 점수의 차이가 그 어절의 점수 최대값과 tolerance 이하로 난다면, 길이가 가장 긴 어절을 선택한다. CohesionProbability에서는 합성명사들은 각각의 요소들보다 낮기 때문에 tolerance를 이용할 수 있다. 

In [5]:
print('tolerance=0.0\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다'))

print('\ntolerance=0.1\nsent = 데이터마이닝을 공부중이다')
print(tokenizer.tokenize('데이터마이닝을 공부중이다', tolerance=0.1))

tolerance=0.0
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부', '중이다']

tolerance=0.1
sent = 데이터마이닝을 공부중이다
['데이터마이닝', '을', '공부중', '이다']


## RegexTokenizer

단어를 추출하지 않아도 기본적으로 토크나이징이 되어야 하는 부분들이 있습니다. 언어의 종류가 바뀌는 부분이다. 

    이것은123이라는숫자

위의 에제에서 물론 숫자와 한글이 합쳐져서 하나의 단어가 되기도 한다. 6.25전쟁이 '6.25', '전쟁'으로 나뉘어진 다음에, 이를 '6.25 - 전쟁'으로 묶는 건 ngram extraction으로 할 수 있다. 

'6.25전쟁'과 같은 경우는 소수이며, 대부분의 경우에는 한글|숫자|영어(라틴)|기호가 바뀌는 지점에서 토크나이징이 되어야 한다. 위의 예제는 적어도 [이것은, 123, 이라는숫자]로 니뉘어져야 한다. 그 다음에 단어 추출에 의하여 [이것, 은, 123, 이라는, 숫자]라고 나뉘어지는 것이 이상적이다.

또한 한국어에서 자음/모음이 단어 중간에 단어의 경계를 구분해주는 역할을 한다 .우리는 문자 메시지를 주고 받을 때 자음으로 이뤄진 이모티콘들로 띄어쓰기를 대신하기도 한다. 

    아이고ㅋㅋ진짜? = [아이고, ㅋㅋ, 진짜, ?]
    
'ㅋㅋ' 덕분에 '아이고'와 '진짜'가 구분이 된다. 또한 'ㅠㅠ'는 'ㅋㅋ'와 함께 붙어있으면 서로 다른 이모티콘으로 구분이 될 수 있다.

    아이고ㅋㅋㅜㅜ진짜? = [아이고, ㅋㅋ, ㅜㅜ, 진짜, ?]
    
이를 분리하는 손쉬운 방법은 'ㅋㅋ'를 찾아내어 앞/뒤에 빈 칸을 하나씩 추가한다. 

    str.replace('ㅋㅋ', ' ㅋㅋ ')

str 하나를 replace 하는 것은 쉽습니다만, 모든 연속된 모음, 혹은 연속된 한글이라는 것은 하나의 str이 아니다. 이와 같이 str에서의 어떤 pattern을 찾아내는 것이 regular expression이다. 

    re.compile('[가-힣]+')
    
위 regular expression은 초/중/종성이 완전한 한국어의 시작부터 끝까지라는 의미이다. 

    re.compile('[ㄱ-ㅎ]+')
    
위 regular expression은 ㄱ부터 ㅎ까지 자음의 범위를 나타낸다. 

RegexTokenizer는 regular extression을 이용하여 언어가 달라지는 순간들을 띄어쓴다. 영어의 경우에는 움라우트가 들어가는 경우들이 있어서 알파벳 뿐 아니라 라틴까지 포함하였다. 

In [6]:
tokenizer = RegexTokenizer()

sents = [
    '이렇게연속된문장은잘리지않습니다만',
    '숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠',
    '띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠'
]

for sent in sents:
    print('   %s\n->%s\n' % (sent, tokenizer.tokenize(sent)))

   이렇게연속된문장은잘리지않습니다만
->['이렇게연속된문장은잘리지않습니다만']

   숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠
->['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']

   띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠
->['띄어쓰기가', '포함되어있으면', '이정보는', '10', '점', '!', '꼭띄워야죠']



RegexTokenizer 역시 flatten=False이면 띄어쓰기 기준으로 토큰을 나눠서 출력한다

In [7]:
tokenizer.tokenize('띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠',flatten=False)

[['띄어쓰기가'], ['포함되어있으면'], ['이정보는', '10', '점', '!', '꼭띄워야죠']]